In [1]:
## AST425 Rebound Simulation
## Author: Chris Simbulan

In [2]:
## Import Libraries -----------------------------------------------------------------------------------------
import numpy as np
import rebound as reb
import copy
from itertools import combinations
import random
import os.path

In [3]:
## Check if log file exists, if not, create a blank text file
if (os.path.exists("log.txt") == False):
    x = np.array([])
    np.savetxt("log.txt", x)

## Check if data file exists, if not, create a blank text file    
if (os.path.exists("data.txt") == False):
    x = np.array([])
    np.savetxt("data.txt", x)
    
## Check if planet file exists, if not, create a blank text file    
if (os.path.exists("planets.txt") == False):
    x = np.array([])
    np.savetxt("planets.txt", x)
    
## Open the log, data and planet files for reading and writing
logfile = open("log.txt", "r+")
datafile = open("data.txt", "r+")
planetfile = open("planets.txt", "r+")

In [4]:
## Create function to setup bodies with initial conditions
def setup():
    sim = reb.Simulation()
    ## Define units, although these are the default
    sim.units = ("AU", "yr", "Msun")
    Mj = 9.542e-4 # Mass of Jupiter
    Mst = 2.856e-4 # Mass of Saturn
    Mnpt = 5.149e-5 # Mass of Neptune
    
    ## Generate five random starting angles
    angles = []
    for i in range(0,5):
        angles.append(random.random()*2*np.pi)
    
    ## Add bodies
    sim.add(m=1.0, id = 0) #Add solar mass planet
    logfile.write("Solar mass star added at center.\n")
    sim.add(m = Mst, a=13.6, e=0, inc = 0, f=angles[0], id = 1)
    logfile.write("Saturn mass planet added 13.6 AU away and {0} degrees.\n".format(angles[0]*180/np.pi))
    sim.add(m = Mst, a=33.3, e=0, inc = 0, f=angles[1], id = 2)
    logfile.write("Saturn mass planet added 33.3 AU away and {0} degrees.\n".format(angles[1]*180/np.pi))
    sim.add(m = Mst, a=65.1, e=0, inc = 0, f=angles[2], id = 3)
    logfile.write("Saturn mass planet added 65.1 AU away and {0} degrees.\n".format(angles[2]*180/np.pi))
    sim.add(m = Mst, a=77.3, e=0, inc = 0, f=angles[3], id = 4)
    logfile.write("Saturn mass planet added 77.3 AU away and {0} degrees.\n".format(angles[3]*180/np.pi))
    sim.add(m = Mst, a=93.0, e=0, inc = 0, f=angles[4], id = 5)
    logfile.write("Saturn mass planet added 93.0 AU away and {0} degrees.\n".format(angles[4]*180/np.pi))
    sim.move_to_com()
    return sim

In [5]:
## Function to merge particles
def mergeParticles(sim, time):
    ## Find two closest particles
    min_d2 = 1e9 # large number
    particles = sim.particles
    for p1, p2 in combinations(particles,2):
        dx = p1.x - p2.x
        dy = p1.y - p2.y
        dz = p1.z - p2.z
        d2 = dx*dx + dy*dy + dz*dz
        if d2<min_d2:
            min_d2 = d2
            cp1 = p1
            cp2 = p2
    
    ## Merge two closest particles
    mergedPlanet = reb.Particle()
    mergedPlanet.m  = cp1.m + cp2.m
    mergedPlanet.x  = (cp1.m*cp1.x  + cp2.m*cp2.x) /mergedPlanet.m
    mergedPlanet.y  = (cp1.m*cp1.y  + cp2.m*cp2.y) /mergedPlanet.m
    mergedPlanet.z  = (cp1.m*cp1.z  + cp2.m*cp2.z) /mergedPlanet.m
    mergedPlanet.vx = (cp1.m*cp1.vx + cp2.m*cp2.vx)/mergedPlanet.m
    mergedPlanet.vy = (cp1.m*cp1.vy + cp2.m*cp2.vy)/mergedPlanet.m
    mergedPlanet.vz = (cp1.m*cp1.vz + cp2.m*cp2.vz)/mergedPlanet.m
    mergedPlanet.id = cp1.id
    id1 = cp1.id
    id2 = cp2.id
    sim.remove(id=id1)
    sim.remove(id=id2)
    sim.add(mergedPlanet)
    
    ## Write to log file
    logfile.write("Planets {0} and {1} have collided and merged ".format(str(id1), str(id2)))
    logfile.write("at t = {0}.\n".format(time))
    print ("Planets {0} and {1} have collided and merged and became Planet {2} ".format(str(id1), str(id2), mergedPlanet.id))
    print("at t = {0}.\n".format(time))

In [ ]:
## Define function to eject particles
def ejectParticle(sim, time):    
    max_d2 = 0.
    for p in sim.particles:
        d2 = p.x*p.x + p.y*p.y + p.z*p.z
        if d2>max_d2:
            max_d2 = d2
            mid = p.id
    sim.remove(id=mid)
        
    if mid != 0:
        logfile.write("Planet {0} has been ejected ".format(str(mid)))
        logfile.write("at t = {0}.\n".format(time))
        print ("Planet {0} has been ejected ".format(str(mid)))
        print("at t = {0}.\n".format(time))

In [ ]:
## Create simulation
sim = setup()
   
## Create array of pointers
ps = sim.particles

## Set distance for collision in AU
Rj = 0.000477894503
Rst = 0.00038925688
sim.exit_min_distance = Rst
## Set distance for ejection in AU
sim.exit_max_distance = 2000.0
   
## Number of outputs
Noutputs = 100000
times = np.linspace(0,1.0e7*2.*np.pi,Noutputs)

#distances = np.zeros((len(ps)-1,Noutputs))
#xs = np.zeros((len(ps),Noutputs))
#ys = np.zeros((len(ps),Noutputs))
    
## Plot the positions of the planets
#%matplotlib inline
#fig = reb.OrbitPlot(sim)
   
## Count number of ejections
NE = 0
   
## Integrate the simulation
for i,time in enumerate(times):
    try:
        sim.integrate(time)
        ## This is to calculate the distance between each adjacent body for plotting purpose
        ## to see if a collision does happen
        
        #for j in range(1, (len(ps) - 1)):
        #    dx = ps[j].x - ps[j+1].x
        #    dy = ps[j].y - ps[j+1].y
        #    dz = ps[j].z - ps[j+1].z
        #    distances[j-1][i] = np.sqrt(dx*dx+dy*dy+dz*dz)
        
    ## If a the particles come too close, merge them
    except reb.Encounter as error:
        mergeParticles(sim, time)     
    ## If a particle reaches past the max distance, treat is as ejected
    except reb.Escape as error:
        ejectParticle(sim, time)
        NE += 1
    if (time >= 1.0e6*2.*np.pi):
        #datafile.write("\nAt time t = {0}: ----------\n\n".format(time))  
        for o in sim.calculate_orbits():
            #print o
            ## Write the semi-major axis, eccentricity, and inclination to data file
            datafile.write("{0}\t{1}\t{2}\t{3}\t{4}\t{5}\t{6}\n".format(time, o.a, o.e,
                                                                             o.inc, o.Omega, o.omega, o.f))        
    #for k in range(sim.N):
    #    xs[sim.particles[k].id,i] = sim.particles[k].x
    #    ys[sim.particles[k].id,i] = sim.particles[k].y
logfile.write("Number of planets at the end of the simulation: {0}\n\n".format(sim.N - 1))
#print("Number of particles at the end of the simulation: %d."%sim.N)
   

## Save the simulation
sim.save("HL_Tau.bin")

print ("1.0e7 orbital periods done")

In [ ]:
## Close all the files
logfile.close()
datafile.close()
planetfile.close()